# Chat Financeiro Autônomo (CrewAI + Databricks)
Este notebook implementa um assistente financeiro interativo. Ele pode analisar ativos específicos (gerando teses de investimento) ou responder perguntas gerais sobre o mercado.

In [ ]:
# 1. Configuração e Instalação
%pip install crewai yfinance langchain-databricks duckduckgo-search matplotlib
dbutils.library.restartPython()

In [ ]:
import os
import json
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from crewai import Agent, Task, Crew, Process
from langchain_databricks import ChatDatabricks
from langchain.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.prompts import PromptTemplate

In [ ]:
# 2. Definição do LLM
models = [
    'databricks-gpt-oss-20b',
    'databricks-gpt-oss-120b',
    'databricks-llama-4-maverick',
    'databricks-gemma-3-12b',
    'databricks-meta-llama-3-1-8b-instruct',
    'databricks-meta-llama-3-3-70b-instruct', # ESCOLHIDO PARA RACIOCÍNIO
    'databricks-gte-large-en',
    'databricks-meta-llama-3-1-405b-instruct'
]

llm_databricks = ChatDatabricks(
    endpoint=models[5],
    temperature=0.1,
    max_tokens=4000
)

In [ ]:
# 3. Ferramentas (Tools)

class FinancialTools:
    @tool("fetch_stock_data")
    def fetch_stock_data(ticker: str):
        """
        Baixa dados históricos (180 dias), calcula RSI/SMA e gera gráfico em /tmp/analysis.png.
        """
        try:
            stock = yf.Ticker(ticker)
            df = stock.history(period="180d")
            if df.empty: return f"Erro: Sem dados para {ticker}."

            # Indicadores
            df['SMA_50'] = df['Close'].rolling(window=50).mean()
            df['SMA_200'] = df['Close'].rolling(window=200).mean()
            delta = df['Close'].diff()
            gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
            loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
            rs = gain / loss
            df['RSI'] = 100 - (100 / (1 + rs))
            
            # Gráfico
            plt.figure(figsize=(10, 6))
            plt.plot(df.index, df['Close'], label='Preço', color='blue')
            plt.plot(df.index, df['SMA_50'], label='SMA 50', color='orange', linestyle='--')
            plt.plot(df.index, df['SMA_200'], label='SMA 200', color='red', linestyle='--')
            plt.title(f'Análise: {ticker}')
            plt.legend(); plt.grid(True)
            plt.savefig("/tmp/analysis.png")
            plt.close()
            
            return f"Preço: {df['Close'].iloc[-1]:.2f}, RSI: {df['RSI'].iloc[-1]:.2f}, SMA50: {df['SMA_50'].iloc[-1]:.2f}"
        except Exception as e: return str(e)

    @tool("search_market_news")
    def search_market_news(query: str):
        """Busca notícias e informações gerais sobre o mercado ou ativos."""
        return DuckDuckGoSearchRun().run(query)

In [ ]:
# 4. Agentes

# Agentes de Análise de Ticker
quant_agent = Agent(
    role='The Quant',
    goal='Análise Técnica (Tendência, RSI, Suporte/Resistência).',
    backstory='Especialista em análise técnica e padrões gráficos.',
    llm=llm_databricks, tools=[FinancialTools.fetch_stock_data]
)

fundamentalist_agent = Agent(
    role='The Fundamentalist',
    goal='Análise de Notícias e Sentimento.',
    backstory='Especialista em macroeconomia e notícias de mercado.',
    llm=llm_databricks, tools=[FinancialTools.search_market_news]
)

portfolio_manager = Agent(
    role='Portfolio Manager',
    goal='Decisão de Investimento (Buy/Sell/Hold).',
    backstory='Chefe do comitê de investimentos.',
    llm=llm_databricks, allow_delegation=True
)

# Agente para Perguntas Gerais
market_scout = Agent(
    role='Market Scout',
    goal='Responder perguntas gerais sobre o mercado financeiro.',
    backstory='Pesquisador ágil que busca as melhores oportunidades e notícias do momento.',
    llm=llm_databricks, tools=[FinancialTools.search_market_news]
)

In [ ]:
# 5. Lógica de Chat e Orquestração

def identify_intent(user_input):
    """Usa o LLM para decidir se é uma análise de ticker ou pergunta geral."""
    prompt = f"""
    Analise a entrada do usuário: "{user_input}"
    
    Se o usuário perguntar sobre uma empresa específica (ex: "Como está a Petrobras", "Vale vale a pena?"), 
    retorne um JSON com "action": "analyze_ticker" e o "ticker" (ex: "PETR4.SA"). 
    Se não souber o ticker, tente inferir (priorize .SA para empresas brasileiras).
    
    Se for uma pergunta geral (ex: "Melhores empresas para investir", "Resumo do mercado hoje"),
    retorne "action": "general_chat" e uma "query" de busca otimizada em inglês.
    
    Exemplo Saída 1: {{"action": "analyze_ticker", "ticker": "PETR4.SA"}}
    Exemplo Saída 2: {{"action": "general_chat", "query": "best stocks to buy brazil today market analysis"}}
    
    Retorne APENAS o JSON.
    """
    response = llm_databricks.invoke(prompt).content
    # Limpeza básica para garantir JSON válido
    try:
        start = response.find('{')
        end = response.rfind('}') + 1
        return json.loads(response[start:end])
    except:
        return {"action": "general_chat", "query": user_input}

def run_ticker_analysis(ticker):
    print(f"\n>>> Iniciando Comitê de Investimentos para: {ticker}...")
    
    t1 = Task(
        description=f"Analise tecnicamente {ticker} (Tendência, RSI, Médias).",
        expected_output="Relatório técnico.",
        agent=quant_agent
    )
    t2 = Task(
        description=f"Busque notícias e sentimento para {ticker}.",
        expected_output="Resumo de notícias e sentimento.",
        agent=fundamentalist_agent
    )
    t3 = Task(
        description=f"Decida (Buy/Sell/Hold) para {ticker} com base nos relatórios.",
        expected_output="Relatório final de investimento em Markdown.",
        agent=portfolio_manager,
        context=[t1, t2]
    )
    
    crew = Crew(agents=[quant_agent, fundamentalist_agent, portfolio_manager], tasks=[t1, t2, t3], verbose=True)
    result = crew.kickoff()
    
    print("\n### RELATÓRIO FINAL ###\n")
    print(result)
    
    if os.path.exists("/tmp/analysis.png"):
        try:
            from IPython.display import Image, display
            display(Image(filename="/tmp/analysis.png"))
        except: pass

def run_general_chat(query):
    print(f"\n>>> Pesquisando sobre: {query}...")
    task = Task(
        description=f"Responda à pergunta do usuário com base em pesquisas de mercado: '{query}'. Use a ferramenta de busca.",
        expected_output="Uma resposta direta e informativa baseada em dados recentes.",
        agent=market_scout
    )
    crew = Crew(agents=[market_scout], tasks=[task], verbose=True)
    result = crew.kickoff()
    print("\n### RESPOSTA ###\n")
    print(result)

def chat_session():
    print("=== Assistente Financeiro IA ===")
    user_input = input("Pergunte sobre uma ação (ex: 'Como está a Petrobras?') ou sobre o mercado: ")
    
    if not user_input: return
    
    intent = identify_intent(user_input)
    print(f"DEBUG: Intenção detectada -> {intent}")
    
    if intent.get("action") == "analyze_ticker":
        run_ticker_analysis(intent.get("ticker"))
    else:
        run_general_chat(intent.get("query", user_input))

In [ ]:
# 6. Executar Chat
# Execute esta célula para interagir
chat_session()